In [7]:
import pandas as pd
import polars as pl
import duckdb as db
import numpy as np
import scipy.stats as st

from sklearn.datasets import load_digits
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import silhouette_score, adjusted_rand_score, normalized_mutual_info_score

import time
import random

from ucimlrepo import fetch_ucirepo 

import matplotlib.pyplot as plt

from urllib.request import urlopen
import xmltodict

# HW2

Overall rules:

- Do not split your answers into separate files. All answers must be in a single jupyter notebook. 
- Refrain from downloading and loading data from a local file unless specifically specified. Obtain all required remote data using the appropriate API.
- Refrain from cleaning data by hand on a spreadsheet. All cleaning must be done programmatically, with each step explained. This is so that I can replicate the procedure deterministically.
- Refrain from using code comments to explain what has been done. Document your steps by writing appropriate markdown cells in your notebook.
- Avoid duplicating code by copying and pasting it from one cell to another. If copying and pasting is necessary, develop a suitable function for the task at hand and call that function.
- When providing parameters to a function, never use global variables. Instead, always pass parameters explicitly and always make use of local variables.
- Document your use of LLM models (ChatGPT, Claude, Code Pilot etc). Either take screenshots of your steps and include them with this notebook, or give me a full log (both questions and answers) in a markdown file named HW2-LLM-LOG.md.

Failure to adhere to these guidelines will result in a 15-point deduction for each infraction.

## Q1

For this question we are going to use [RT-IoT2022](https://archive.ics.uci.edu/dataset/942/rt-iot2022) dataset from [UCI](https://archive.ics.uci.edu/). We have seen several clustering algorithms, and also some measures of quality for clusters in the lectures. Use all you have learned and compare the clustering algorithms we have learnt using the quality measures we have seen in the class.  You must write a detailed comparison analysis to evaluate which model behaves better based on the results you will obtain. One part of your analysis should also include how much time it takes to train and run the models.

In [3]:
rt_iot2022 = fetch_ucirepo(id=942) 
  
X = rt_iot2022.data.features 
y = rt_iot2022.data.targets
y = y.to_numpy().reshape(-1) 

Let us first process the data. We must convert the categorical data into numerical data. At the bottom, I am going to shuffle the dataset. We are going to need it later when we split the data.

In [10]:
categorical_columns = X.select_dtypes(include=['object', 'category']).columns
numerical_columns = X.select_dtypes(include=['number']).columns

# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_columns),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns)
    ]
)

X_processed = preprocessor.fit_transform(X)
random.shuffle(X_processed)

This is the main engine of our experiments. It takes a part of the data, clusters it and then evaluates the quality of the resulting clusters.

In [ ]:
def experiment(model, X_part, y_part):
    result = {"Silhouette Score": [],
              "Adjusted Rand Index": [],
              "Normalized Mutual Information": [],
              "Runtime (s)": []}
              
    start_time = time.time()
    labels = model.fit_predict(X_part)
    end_time = time.time()
    
    result['Silhouette Score'].append(silhouette_score(X_part, labels))
    result['Adjusted Rand Index'].append(adjusted_rand_score(y_part, labels))
    result['Normalized Mutual Information'].append(normalized_mutual_info_score(y_part, labels))
    result['Runtime (s)'].append(end_time - start_time)
   
    return result

I am going to split the data into 3 pieces. We already shuffled the data before. So, the split creates 3 random pieces of data. I am going to run the experiment on each piece. Then we are going to determine the confidence intervals for each measure.

In [37]:
models = [
    {"name": "K-Means",
     "model": KMeans(n_clusters=3, random_state=42),
     "results": []},
    {"name": "DBSCAN",
     "model": DBSCAN(eps=0.5, min_samples=5),
     "results": []},
    {"name": "Agglomerative",
     "model": AgglomerativeClustering(n_clusters=3),
     "results": []}
]

Function to calculate 95% confidence intervals:

In [ ]:
def confidence_interval(data, confidence=0.95):
    n = len(data)
    mean = np.mean(data)
    std_err = np.std(data, ddof=1) / np.sqrt(n)
    margin = std_err * t.ppf((1 + confidence) / 2, n - 1)
    return mean, margin

In [ ]:
indices = [int(x) for x in np.linspace(0,1,4)*X_processed.shape[0]]
for model in models:
    tmp = []
    for i in range(3):
        X_part = X_processed[indices[i]:indices[i+1]]
        y_part = y[indices[i]:indices[i+1]]
        tmp.append(experiment(model['model'], X_part, y_part))
    mean
    


## Q2

For this question we are going to use MNIST digits dataset. We have analyzed 3 classification algorithms so far. These are

1. K-NN
2. SVM
3. Logistric Regression

Use these algorithms to obtain 3 classification models on the dataset. Then use an appropriate cross-validation scheme to test the quality of the models and compare. You must write a detailed comparison analysis to evaluate which model behaves better based on the results you will obtain. One part of your analysis should also include how much time it takes to train and run the models.

In [7]:
digits = load_digits()
digits_X = digits['data']
digits_y = digits['target']

## Q3

For this homework, we are going to use the [data warehouse](https://clerk.house.gov/Votes/) for the [US House of Representatives](https://www.house.gov/). The data server has data on each vote going back to 1990. The voting information is in XML format. For example, the code below pulls the data for the 71st roll call from 2024 Congress.

In [10]:
with urlopen('https://clerk.house.gov/evs/2024/roll071.xml') as url:
    raw = xmltodict.parse(url.read())
raw

{'rollcall-vote': {'vote-metadata': {'majority': 'R',
   'congress': '118',
   'session': '2nd',
   'committee': 'U.S. House of Representatives',
   'rollcall-num': '71',
   'legis-num': 'H R 2799',
   'vote-question': 'On Agreeing to the Amendment',
   'amendment-num': '4',
   'amendment-author': 'Wagner of Missouri Part B Amendment No. 4',
   'vote-type': 'RECORDED VOTE',
   'vote-result': 'Agreed to',
   'action-date': '7-Mar-2024',
   'action-time': {'@time-etz': '14:28', '#text': '2:28 PM'},
   'vote-desc': None,
   'vote-totals': {'totals-by-party-header': {'party-header': 'Party',
     'yea-header': 'Ayes',
     'nay-header': 'Noes',
     'present-header': 'Answered “Present”',
     'not-voting-header': 'Not Voting'},
    'totals-by-party': [{'party': 'Republican',
      'yea-total': '213',
      'nay-total': '0',
      'present-total': '0',
      'not-voting-total': '8'},
     {'party': 'Democratic',
      'yea-total': '57',
      'nay-total': '154',
      'present-total': '0',

1. Obtain all of the data from 2010 to 2024 (inclusive, all years, all calls). You may save a local copy, and push the local copy onto your github repo.
2. Construct a similarity measure function for a pair of legislators. The function should return the number of times both legislators voted the same way and the number of sessions both attended.
3. Find the legislators whose voting records are the most similar to: (i) Matt Gaetz and (ii) Alexandria Ocasio-Cortez.
4. Using this similarity measure, do a hiearchical clustering for legislators, and analyze the result.
5. Construct a function that takes a year as an input and returns a 2x2 table couting the number of times Democrats voted with other Democrats, the number of times Democrats voted with Republicans, the number of times Republicans voted with Democrats, and the number of times Republicans voted with other Republicans in that year.
6. Analyze the table for each year since 2010 using the $\chi^2$-statistic, and evaluate the polarization in the House of representatives. In this context, explain what a lower or higher $\chi^2$-metric means. Explain whether the polarization in the House of representatives increased or decreased over the years?

## Q4

For this question, we are going to use the following dataset.

In [11]:
credit = pd.read_csv('https://storage.googleapis.com/download.tensorflow.org/data/creditcard.csv')
credit.head(10)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
5,2.0,-0.425966,0.960523,1.141109,-0.168252,0.420987,-0.029728,0.476201,0.260314,-0.568671,...,-0.208254,-0.559825,-0.026398,-0.371427,-0.232794,0.105915,0.253844,0.081080,3.67,0
6,4.0,1.229658,0.141004,0.045371,1.202613,0.191881,0.272708,-0.005159,0.081213,0.464960,...,-0.167716,-0.270710,-0.154104,-0.780055,0.750137,-0.257237,0.034507,0.005168,4.99,0
7,7.0,-0.644269,1.417964,1.074380,-0.492199,0.948934,0.428118,1.120631,-3.807864,0.615375,...,1.943465,-1.015455,0.057504,-0.649709,-0.415267,-0.051634,-1.206921,-1.085339,40.80,0
8,7.0,-0.894286,0.286157,-0.113192,-0.271526,2.669599,3.721818,0.370145,0.851084,-0.392048,...,-0.073425,-0.268092,-0.204233,1.011592,0.373205,-0.384157,0.011747,0.142404,93.20,0
9,9.0,-0.338262,1.119593,1.044367,-0.222187,0.499361,-0.246761,0.651583,0.069539,-0.736727,...,-0.246914,-0.633753,-0.120794,-0.385050,-0.069733,0.094199,0.246219,0.083076,3.68,0


1. Write a regression model that predicts the Amount from the other variables. Assess the quality of the model using an appropriate measure.
2. Write a logistic regression model that predicts the Class from other variables. Assess the quality of the model using an appropriate measure.
3. The quality of the model seem very good, but in reality it doesn't work. Why? What is the problem? Offer a solution, implement and test it. Did it really solve our problem? Explain.
4. Try other supervised classification models we have learned. Are they susceptible to the same problem as before? Explain. Offer a solution, implement and test it. Did it really solve our problem? Explain.